# TP3 : Neo4j

In [50]:
from neo4j import GraphDatabase

## Question 1 

### Exemple des universités

In [51]:
# Database Credentials

uri             = "bolt://localhost:7687"

userName        = "neo4j"

password        = "test"

 

# Connect to the neo4j database server

graphDB_Driver  = GraphDatabase.driver(uri, auth=(userName, password))

 

# CQL to query all the universities present in the graph

cqlNodeQuery          = "MATCH (x:university) RETURN x"

 

# CQL to query the distances from Yale to some of the other Ivy League universities

cqlEdgeQuery          = "MATCH (x:university {name:'Yale University'})-[r]->(y:university) RETURN y.name,r.miles"

 

# CQL to create a graph containing some of the Ivy League universities

cqlCreate = """CREATE (cornell:university { name: "Cornell University"}),

(yale:university { name: "Yale University"}),

(princeton:university { name: "Princeton University"}),

(harvard:university { name: "Harvard University"}),

 

(cornell)-[:connects_in {miles: 259}]->(yale),

(cornell)-[:connects_in {miles: 210}]->(princeton),

(cornell)-[:connects_in {miles: 327}]->(harvard),

 

(yale)-[:connects_in {miles: 259}]->(cornell),

(yale)-[:connects_in {miles: 133}]->(princeton),

(yale)-[:connects_in {miles: 133}]->(harvard),

 

(harvard)-[:connects_in {miles: 327}]->(cornell),

(harvard)-[:connects_in {miles: 133}]->(yale),

(harvard)-[:connects_in {miles: 260}]->(princeton),

 

(princeton)-[:connects_in {miles: 210}]->(cornell),

(princeton)-[:connects_in {miles: 133}]->(yale),

(princeton)-[:connects_in {miles: 260}]->(harvard)"""

# CQL to delete all

cqlDeleteAll = "MATCH (n) DETACH DELETE n"

# Execute the CQL query

with graphDB_Driver.session() as graphDB_Session:
    
    # Delete all nodes and relationships
    graphDB_Session.run(cqlDeleteAll)

    # Create nodes

    graphDB_Session.run(cqlCreate)

   

    # Query the graph    

    nodes = graphDB_Session.run(cqlNodeQuery)

   

    print("List of Ivy League universities present in the graph:")

    for node in nodes:

        print(node)

 

    # Query the relationships present in the graph

    nodes = graphDB_Session.run(cqlEdgeQuery)

   

    print("Distance from Yale University to the other Ivy League universities present in the graph:")

    for node in nodes:

        print(node)

List of Ivy League universities present in the graph:
<Record x=<Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:16' labels=frozenset({'university'}) properties={'name': 'Cornell University'}>>
<Record x=<Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:17' labels=frozenset({'university'}) properties={'name': 'Yale University'}>>
<Record x=<Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:18' labels=frozenset({'university'}) properties={'name': 'Princeton University'}>>
<Record x=<Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:19' labels=frozenset({'university'}) properties={'name': 'Harvard University'}>>
Distance from Yale University to the other Ivy League universities present in the graph:
<Record y.name='Princeton University' r.miles=133>
<Record y.name='Harvard University' r.miles=133>
<Record y.name='Cornell University' r.miles=259>


### Récupérer tous les noeuds

In [52]:
def get_all_nodes(database):
    """Permet d'afficher tous les noeuds de la database en entrée"""
    with graphDB_Driver.session() as graphDB_Session:
        cqlGetAllNodes = f"MATCH (n:{database}) RETURN n"
        nodes = graphDB_Session.run(cqlGetAllNodes)
        for node in nodes :
            print(node)
        
get_all_nodes('university')


<Record n=<Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:16' labels=frozenset({'university'}) properties={'name': 'Cornell University'}>>
<Record n=<Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:17' labels=frozenset({'university'}) properties={'name': 'Yale University'}>>
<Record n=<Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:18' labels=frozenset({'university'}) properties={'name': 'Princeton University'}>>
<Record n=<Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:19' labels=frozenset({'university'}) properties={'name': 'Harvard University'}>>


### Récupérer un noeud par id

In [53]:
def get_node_by_id(database, id):
    """Renvoie le noeud correspondant à l'id en paramètre pour la database en entrée"""
    with graphDB_Driver.session() as graphDB_Session:
        cqlGetNode = f"MATCH (n:{database}) WHERE id(n) = {id} RETURN n"
        print(cqlGetNode)
        nodes = graphDB_Session.run(cqlGetNode)
        for node in nodes :
            print(node)

get_node_by_id('university', '15')

MATCH (n:university) WHERE id(n) = 15 RETURN n


### Récupérer des noeuds par leur label

In [54]:
def get_nodes_by_label(database, label):
    """Renvoie les noms comportant le label en paramètre dans leur labels, pour la database en entrée"""
    with graphDB_Driver.session() as graphDB_Session:
        cqlGetNodesByLabel = f"MATCH (n:{database}) WHERE '{label}' IN labels(n) RETURN n"
        print(cqlGetNodesByLabel)
        nodes = graphDB_Session.run(cqlGetNodesByLabel)
        for node in nodes :
            print(node)

get_nodes_by_label('university', 'university')

MATCH (n:university) WHERE 'university' IN labels(n) RETURN n
<Record n=<Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:16' labels=frozenset({'university'}) properties={'name': 'Cornell University'}>>
<Record n=<Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:17' labels=frozenset({'university'}) properties={'name': 'Yale University'}>>
<Record n=<Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:18' labels=frozenset({'university'}) properties={'name': 'Princeton University'}>>
<Record n=<Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:19' labels=frozenset({'university'}) properties={'name': 'Harvard University'}>>


### Récupérer les relations

In [55]:
def get_all_relationships(database):
    """Permet d'afficher toutes les relations présentes dans la database en entrée"""
    with graphDB_Driver.session() as session:
        cqlGetAllRelationships = session.run(f"MATCH (n:{database})-[r]->(m) RETURN r")
        for r in cqlGetAllRelationships:
            print(r)
            
get_all_relationships('university')

<Record r=<Relationship element_id='5:e24570d5-879c-4ca5-b2f2-08d7b506c520:49' nodes=(<Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:16' labels=frozenset() properties={}>, <Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:18' labels=frozenset() properties={}>) type='connects_in' properties={'miles': 210}>>
<Record r=<Relationship element_id='5:e24570d5-879c-4ca5-b2f2-08d7b506c520:50' nodes=(<Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:16' labels=frozenset() properties={}>, <Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:19' labels=frozenset() properties={}>) type='connects_in' properties={'miles': 327}>>
<Record r=<Relationship element_id='5:e24570d5-879c-4ca5-b2f2-08d7b506c520:48' nodes=(<Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:16' labels=frozenset() properties={}>, <Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:17' labels=frozenset() properties={}>) type='connects_in' properties={'miles': 259}>>
<Record r=<Relationship e

### Récupérer les relations d'un noeud

In [56]:
def get_node_relationships(database, node) :
    """Permet d'afficher toutes les relations du noeud dont le nom est en entrée pour la database en entrée"""
    with graphDB_Driver.session() as graphDB_Session:
        cqlGetNodeRelationships = f"MATCH (n:{database}" + "{name:'" + node + "'}" + f")-[r]->(m:{database}) RETURN r"
        result = graphDB_Session.run(cqlGetNodeRelationships)
        for r in result :
            print(r)

get_node_relationships('university', 'Princeton University')

<Record r=<Relationship element_id='5:e24570d5-879c-4ca5-b2f2-08d7b506c520:59' nodes=(<Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:18' labels=frozenset() properties={}>, <Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:19' labels=frozenset() properties={}>) type='connects_in' properties={'miles': 260}>>
<Record r=<Relationship element_id='5:e24570d5-879c-4ca5-b2f2-08d7b506c520:58' nodes=(<Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:18' labels=frozenset() properties={}>, <Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:17' labels=frozenset() properties={}>) type='connects_in' properties={'miles': 133}>>
<Record r=<Relationship element_id='5:e24570d5-879c-4ca5-b2f2-08d7b506c520:57' nodes=(<Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:18' labels=frozenset() properties={}>, <Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:16' labels=frozenset() properties={}>) type='connects_in' properties={'miles': 210}>>


### Récupérer une relation par son id

In [64]:
def get_relationship_by_id(database, id):
    """Renvoie la relation correspondant à l'id en paramètre pour la database en entrée"""
    with graphDB_Driver.session() as graphDB_Session:
        cqlGetR = f"MATCH (n:{database})-[r]->(m) WHERE id(r) = {id} RETURN r"
        all_r = graphDB_Session.run(cqlGetR)
        for r in all_r :
            print(r)

get_relationship_by_id('university', '58')

<Record r=<Relationship element_id='5:e24570d5-879c-4ca5-b2f2-08d7b506c520:58' nodes=(<Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:18' labels=frozenset() properties={}>, <Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:17' labels=frozenset() properties={}>) type='connects_in' properties={'miles': 133}>>


### Récupérer des relations par leur type

In [65]:
def get_relationships_by_type(database, type):
    """Renvoie les relations du type en paramètre pour la database en entrée"""
    with graphDB_Driver.session() as graphDB_Session:
        cqlGetR = f"MATCH (n:{database})-[r]->(m) WHERE type(r) = '{type}' RETURN r"
        print(cqlGetR)
        all_r = graphDB_Session.run(cqlGetR)
        for r in all_r :
            print(r)

get_relationships_by_type('university', 'connects_in')

MATCH (n:university)-[r]->(m) WHERE type(r) = 'connects_in' RETURN r
<Record r=<Relationship element_id='5:e24570d5-879c-4ca5-b2f2-08d7b506c520:49' nodes=(<Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:16' labels=frozenset() properties={}>, <Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:18' labels=frozenset() properties={}>) type='connects_in' properties={'miles': 210}>>
<Record r=<Relationship element_id='5:e24570d5-879c-4ca5-b2f2-08d7b506c520:50' nodes=(<Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:16' labels=frozenset() properties={}>, <Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:19' labels=frozenset() properties={}>) type='connects_in' properties={'miles': 327}>>
<Record r=<Relationship element_id='5:e24570d5-879c-4ca5-b2f2-08d7b506c520:48' nodes=(<Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:16' labels=frozenset() properties={}>, <Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:17' labels=frozenset() properties={}>) typ

### Créer un noeud

In [67]:
def create_node(label,properties):
    """Permet de créer un noeud avec le label et les propriétés voulues"""
    with graphDB_Driver.session() as session:
        session.run(f"CREATE (:{label} {properties})")

create_node('university',"{name:'Annecy University'}")
# On vérifie que le noeud a bien été créé
get_all_nodes('university')

<Record n=<Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:0' labels=frozenset({'university'}) properties={'name': 'Annecy University'}>>
<Record n=<Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:1' labels=frozenset({'university'}) properties={'name': 'Annecy University'}>>
<Record n=<Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:16' labels=frozenset({'university'}) properties={'name': 'Cornell University'}>>
<Record n=<Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:17' labels=frozenset({'university'}) properties={'name': 'Yale University'}>>
<Record n=<Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:18' labels=frozenset({'university'}) properties={'name': 'Princeton University'}>>
<Record n=<Node element_id='4:e24570d5-879c-4ca5-b2f2-08d7b506c520:19' labels=frozenset({'university'}) properties={'name': 'Harvard University'}>>
